# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [13]:
# parece que existem valores nulos que não fazem muito sentido, ja que é possivel ter números decimais no campo, vamos excluir os 0.
df = df.dropna()
#vamos remover algumas colunas. 
df = df.drop(['Unnamed: 0','data_ref','id_cliente'], axis=1)

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [14]:
df['qtd_filhos'].value_counts()

0     7887
1     2973
2     1358
3      183
4       17
7        5
14       2
5        2
Name: qtd_filhos, dtype: int64

In [15]:
df['qt_pessoas_residencia'].value_counts()

2.0     6469
3.0     2486
1.0     1975
4.0     1292
5.0      179
6.0       18
9.0        5
15.0       2
7.0        1
Name: qt_pessoas_residencia, dtype: int64

In [19]:
reg_df_full = smf.ols('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + C(qt_pessoas_residencia) + 1', data = df).fit()
reg_df_full.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     203.5
Date:                Mon, 22 Apr 2024   Prob (F-statistic):               0.00
Time:                        09:50:35   Log-Likelihood:                -13559.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12392   BIC:                         2.745e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            6.8777      0.145     47.526      0.000       6.594       7.161
sexo[T.M]                            0.7878      0.015     53.664      0.000       0.759       0.817
posse_de_veiculo[T.True]             0.0436      0.014      3.079      0.002       0.016       0.071
posse_de_imovel[T.True]              0.0828      0.014      5.907      0.000       0.055       0.110
C(qtd_filhos)[T.1]                  -0.2501      0.112     -2.239      0.025      -0.469      -0.031
C(qtd_filhos)[T.2]                  -0.5697      0.223     -2.559      0.011      -1.006      -0.133
C(qtd_filhos)[T.3]                  -0.5560      0.486     -1.144      0.253      -1.509       0.397
C(qtd_filhos)[T.4]                   0.3446      0.439      0.785      0.432      -0.516       1.205
C(qtd_filhos)[T.5]                   0.2740      0.407      0.674      0.501      -0.523       1.071
C(qtd_filhos)[T.7]                   0.1576      0.171      0.923      0.356      -0.177       0.492
C(qtd_filhos)[T.14]                 -0.3275      0.256     -1.281      0.200      -0.829       0.174
tipo_renda[T.Bolsista]               0.2259      0.241      0.937      0.349      -0.247       0.699
tipo_renda[T.Empresário]             0.1547      0.015     10.359      0.000       0.125       0.184
tipo_renda[T.Pensionista]           -0.2891      0.243     -1.190      0.234      -0.765       0.187
tipo_renda[T.Servidor público]       0.0551      0.022      2.476      0.013       0.011       0.099
educacao[T.Pós graduação]            0.1038      0.159      0.652      0.515      -0.208       0.416
educacao[T.Secundário]              -0.0130      0.072     -0.181      0.856      -0.154       0.128
educacao[T.Superior completo]        0.0938      0.072      1.298      0.194      -0.048       0.235
educacao[T.Superior incompleto]     -0.0420      0.078     -0.540      0.589      -0.195       0.110
estado_civil[T.Separado]             0.3176      0.112      2.831      0.005       0.098       0.537
estado_civil[T.Solteiro]             0.2556      0.110      2.319      0.020       0.040       0.472
estado_civil[T.União]               -0.0341      0.025     -1.352      0.176      -0.083       0.015
estado_civil[T.Viúvo]                0.3535      0.117      3.017      0.003       0.124       0.583
tipo_residencia[T.Casa]             -0.0424      0.054     -0.784      0.433      -0.149       0.064
tipo_residencia[T.Com os pais]      -0.0233      0.060     -0.387      0.699      -0.142       0.095
tipo_residencia[T.Comunitário]      -0.1177      0.106     -1.110      0.267      -0.326       0.090
tipo_residencia[T.Estúdio]           0.0651      0.099      0.655      0.512      -0.130       0.260
tipo_residencia[T.Governamental]    -0.0546      0.06

Vamos remover a variavel Educação pois ela apresenta diversos Dummies com baixa correlação( alto P-value)

In [26]:
reg_df_one = smf.ols('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda + estado_civil + tipo_residencia + idade + tempo_emprego + C(qt_pessoas_residencia) + 1', data = df).fit()
reg_df_one.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     227.4
Date:                Mon, 22 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:52:49   Log-Likelihood:                -13592.
No. Observations:               12427   AIC:                         2.725e+04
Df Residuals:                   12396   BIC:                         2.748e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            6.9120      0.127     54.498      0.000       6.663       7.161
sexo[T.M]                            0.7804      0.015     53.150      0.000       0.752       0.809
posse_de_veiculo[T.True]             0.0525      0.014      3.713      0.000       0.025       0.080
posse_de_imovel[T.True]              0.0855      0.014      6.091      0.000       0.058       0.113
C(qtd_filhos)[T.1]                  -0.2344      0.112     -2.094      0.036      -0.454      -0.015
C(qtd_filhos)[T.2]                  -0.5553      0.223     -2.489      0.013      -0.993      -0.118
C(qtd_filhos)[T.3]                  -0.5550      0.487     -1.139      0.255      -1.510       0.400
C(qtd_filhos)[T.4]                   0.3103      0.440      0.705      0.481      -0.552       1.173
C(qtd_filhos)[T.5]                   0.2453      0.408      0.602      0.547      -0.554       1.044
C(qtd_filhos)[T.7]                   0.1372      0.171      0.802      0.422      -0.198       0.473
C(qtd_filhos)[T.14]                 -0.3457      0.256     -1.349      0.177      -0.848       0.157
tipo_renda[T.Bolsista]               0.3033      0.242      1.256      0.209      -0.170       0.777
tipo_renda[T.Empresário]             0.1646      0.015     11.041      0.000       0.135       0.194
tipo_renda[T.Pensionista]           -0.2278      0.243     -0.936      0.349      -0.705       0.249
tipo_renda[T.Servidor público]       0.0724      0.022      3.265      0.001       0.029       0.116
estado_civil[T.Separado]             0.3132      0.112      2.785      0.005       0.093       0.534
estado_civil[T.Solteiro]             0.2490      0.110      2.253      0.024       0.032       0.466
estado_civil[T.União]               -0.0371      0.025     -1.469      0.142      -0.087       0.012
estado_civil[T.Viúvo]                0.3432      0.117      2.923      0.003       0.113       0.573
tipo_residencia[T.Casa]             -0.0354      0.054     -0.653      0.514      -0.142       0.071
tipo_residencia[T.Com os pais]      -0.0162      0.061     -0.268      0.789      -0.135       0.102
tipo_residencia[T.Comunitário]      -0.0746      0.106     -0.704      0.482      -0.282       0.133
tipo_residencia[T.Estúdio]           0.0831      0.100      0.834      0.404      -0.112       0.278
tipo_residencia[T.Governamental]    -0.0554      0.066     -0.838      0.402      -0.185       0.074
C(qt_pessoas_residencia)[T.2.0]      0.2621      0.110      2.372      0.018       0.046       0.479
C(qt_pessoas_residencia)[T.3.0]      0.5472      0.219      2.494      0.013       0.117       0.977
C(qt_pessoas_residencia)[T.4.0]      0.9095      0.326      2.788      0.005       0.270       1.549
C(qt_pessoas_residencia)[T.5.0]      0.7685      0.56

### O modelo mudou pouco, se formos bem criteriosos podemos dizer que piorou por conta de um AIC mias alto e um menor R²

Vamos revomer as variaveis pouco importantes para o modelo. (qtd_filhos, tipo_renda, educacao, estado_civill, tipo_residencia, qt_pessoas_residencia

In [25]:
reg_df_aju = smf.ols('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + idade + tempo_emprego + 1', data = df).fit()
reg_df_aju.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Mon, 22 Apr 2024   Prob (F-statistic):               0.00
Time:                        09:57:25   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    7.2550      0.032    228.981      0.000       7.193       7.317
sexo[T.M]                    0.7700      0.015     52.851      0.000       0.741       0.799
posse_de_veiculo[T.True]     0.0579      0.014      4.116      0.000       0.030       0.085
posse_de_imovel[T.True]      0.0880      0.014      6.379      0.000       0.061       0.115
idade                        0.0044      0.001      5.871      0.000       0.003       0.006
tempo_emprego                0.0611      0.001     59.199      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.248   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.536   Jarque-Bera (JB):                1.221
Skew:                           0.022   Prob(JB):                        0.543
Kurtosis:                       3.019   Cond. No.                         209.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### O  modelo ficou com as variáveis mais significantes, mas o AIC e o R² e R² ajustado não melhoraram, mas o modelo esta mais eficiente. 
